In [ ]:
# Import required libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random
from gym import Env, spaces
import time

# Environment Definition
class Experiment_Scope(Env):
    metadata = {"render_modes": ["rgb_array"]} # Metadata

    def __init__(self, observation_shape: tuple = (902, 600, 1), radius_interval: tuple = (10, 50), intensity_interval: tuple = (0, 5), number_of_circles: int = 1, go_cue_interval: tuple = (20, 41)):
        """
            Initializes the Experiment_Scope object.

            Parameters:
            - observation_shape (tuple): Shape of the observation (default: (902, 600, 1))
            - radius_interval (tuple): Interval for circle radius (default: (10, 50))
            - intensity_interval (tuple): Interval for circle intensity (default: (0, 5))
            - number_of_circles (int): Number of circles (default: 1)
            - cue_interval (tuple): Interval for go cue time (default: (20, 41))
        """
        super(Experiment_Scope, self).__init__()

        self.observation_shape = observation_shape
        self.observation_space = spaces.Dict(
            {
                "decision_var": spaces.Box(-100, 100, shape=(1, ), dtype=int)
            }
        )

        self.action_space = spaces.Discrete(3)

        self._action_to_value = {
            1 : -5, # Left
            0 : 0, # No Op
            2 : +5, # Right
        }

        self.render_mode = 'rgb_array'
        self.screen = np.zeros(self.observation_shape) + 255
        self.screen_part = ""
        self.intensity_vector = None

        self.decision_var = 0
        self.is_done = False
        self.time_step = 1
        self.go_cue_time = np.random.randint(go_cue_interval[0], go_cue_interval[1])
        self.cue_played = False

        self.radius_interval = radius_interval
        self.intensity_interval = intensity_interval
        self.number_of_circles = number_of_circles
        self.cue_interval = go_cue_interval

    def _get_obs(self) -> dict:
        """
            Returns a dictionary containing the observation variables.

            Returns:
                dict: A dictionary containing the observation variables.
                    - 'decision_var': The value of the decision variable.
                    - 'cue_played': Whether cue played or not.
        """
        return {'decision_var': self.decision_var,
                'cue_played': self.cue_played}

    def _get_info(self) -> dict:
        """
            Returns a dictionary containing information about the object.

            Returns:
                dict: A dictionary with the following keys:
                    - 'is_done': A boolean indicating whether the task is done or not.
                    - 'time_step': An integer representing the time step of the task.
        """
        return {'is_done': self.is_done, "time_step": self.time_step}

    def _generate_image(self) -> tuple:
        """
            Generates an image with circles and Gabor effects on different parts of the screen.

            Returns:
                tuple: A tuple containing the generated image, the screen part where the circles are located,
                    and the intensities of each screen part.
        """
        width = self.observation_shape[0]
        height = self.observation_shape[1]
        number_of_circles = self.number_of_circles
        intensity_interval = self.intensity_interval
        radius_interval = self.radius_interval
        image = np.zeros((height, width, 1), dtype=np.uint8) + 255
        parts = [1, 2, 3]
        screen_intensities = np.zeros((3, ))
        left_circles = 0
        right_circles = 0
        middle_circles = 0
        color = 0
        thickness = 1
        for _ in range(number_of_circles):
            intensity = np.random.randint(intensity_interval[0], intensity_interval[1])
            int_tmp = intensity * 25
            intensity = 255 - (2 ** (2 * intensity) - 1)
            radius = np.random.randint(radius_interval[0], radius_interval[1])
            image_part = random.choice(parts)
            parts.remove(image_part)
            if image_part == 1:
                # Left Screen
                left_circles += 1
                center_x = np.random.randint(radius, (width-2)//3 - 1 - radius)
                center_y = np.random.randint(radius, height - 1 - radius)
                screen_intensities[0] = int_tmp

            elif image_part == 2:
                # Middle Screen
                middle_circles += 1
                center_x = np.random.randint((width-2) // 3 + radius, 2 * (width-2) // 3 - 1 - radius)
                center_y = np.random.randint(radius, height - 1 - radius)
                screen_intensities[1] = int_tmp

            else:
                # Right Screen
                right_circles += 1
                center_x = np.random.randint(2 * (width-2) // 3  + radius, (width-2) - radius)
                center_y = np.random.randint(radius, height - 1 - radius)
                screen_intensities[2] = int_tmp

            cv2.circle(image, (center_x, center_y), radius, intensity, -1)
            # Gabor Effect
            start_point = (center_x, center_y - radius)
            end_point = (center_x - radius, center_y)
            cv2.line(image, start_point, end_point, 255, thickness + 5)

            start_point = (center_x + radius, center_y)
            end_point = (center_x, center_y + radius)
            cv2.line(image, start_point, end_point, 255, thickness + 5)

            start_point = (center_x + int(np.sqrt(2) / 2 * radius), center_y - int(np.sqrt(2) / 2 * radius))
            end_point = (center_x - int(np.sqrt(2) / 2 * radius), center_y + int(np.sqrt(2) / 2 * radius))
            cv2.line(image, start_point, end_point, 255, thickness + 5)

        # Screen Split lines
        start_point1 = ((width-2) // 3, 0)
        end_point1 = ((width-2) // 3, 599)
        start_point2 = (2 * (width-2) // 3, 0)
        end_point2 = (2 * (width-2) // 3, 599)

        cv2.line(image, start_point1, end_point1, color, thickness)
        cv2.line(image, start_point2, end_point2, color, thickness)

        screen_part = None
        if right_circles > 0 and (left_circles == 0 and middle_circles == 0):
            screen_part = "right"

        elif left_circles > 0 and (right_circles == 0 and middle_circles == 0):
            screen_part = "left"

        elif middle_circles > 0 and (right_circles == 0 and left_circles == 0):
            screen_part = "middle"

        elif right_circles > 0 and left_circles > 0 and middle_circles == 0:
            screen_part = "right-left"

        elif middle_circles > 0 and left_circles > 0 and right_circles == 0:
            screen_part = "middle-left"

        elif right_circles > 0 and middle_circles > 0 and left_circles == 0:
            screen_part = "right-middle"

        elif right_circles > 0 and left_circles > 0 and middle_circles > 0:
            screen_part = "right-middle-left"

        else:
            screen_part = ""


        return (image, screen_part, screen_intensities)

    def reset(self, seed=None, options=None):
        """
            Resets the environment to its initial state.

            Args        seed (int): Optional. The random seed used for generating random numbers.
                options (dict): Optional. Additional options for resetting the environment.

            Returns:
                    observation: The initial observation of the environment.
                    info: Additional information about the environment's state.

            Raises:
                Any exceptions that may occur during the reset process.

            Notes:
                - This method should be called before starting a new episode or when the environment needs to be reset.
                - The `seed` parameter can be used to reproduce the same sequence of random numbers for consistent results.
                - The `options` parameter can be used to pass any additional configuration options specific to the environment.

        """
        super().reset(seed=seed)

        self.screen, self.screen_part, self.intensity_vector = self._generate_image()

        self.decision_var = 0
        self.is_done = False
        self.time_step = 1
        self.go_cue_time = np.random.randint(self.cue_interval[0], self.cue_interval[1])

        observation = self._get_obs()
        info = self._get_info()

        return observation, info

    def step(self, action):
        """
            Executes a single step in the decision-making process.

            Args:
                action (int): The action to take.

            Returns:
                tuple: A tuple containing the observation, reward, and info.

            Raises:
                None

        """
        self.decision_var += self._action_to_value[action]

        if self.time_step >= self.go_cue_time:
            self.cue_played = True

        if self.decision_var >= 100:
            # Right Decision
            self.is_done = True
            if not self.go_cue_time:
                # Make decision before go cue sound
                reward = -300
            else:
                # Make decision after go cue sound
                if 'right' in self.screen_part:
                    reward = 200 - (self.time_step - self.go_cue_time)
                else:
                    reward = -100

        elif self. decision_var <= -100:
            # Left Decision
            self.is_done = True
            if not self.cue_played:
                # Make decision before go cue sound
                reward = -300
            else:
                # Make decision after go cue sound
                if 'left' in self.screen_part:
                    reward = 200 - (self.time_step - self.go_cue_time)
                else:
                    reward = -100

        elif self.time_step >= 200:
            # Middle or No Stimulus
            self.is_done = True
            if self.screen_part == "" or "middle" in self.screen_part:
                reward = 200
            else:
                reward = -100

        else:
            reward = -1
            self.time_step += 1

        observation = self._get_obs()
        info = self._get_info()

        return observation, reward, info

    def render(self):
        """
            Renders the screen.

            Returns:
                The rendered screen.
        """
        return self.screen


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import PIL.Image as Image
import gym
import random

from gym import Env, spaces
import time

font = cv2.FONT_HERSHEY_COMPLEX_SMALL

class ChopperScope(Env):
    def __init__(self):
        super(ChopperScope, self).__init__()

        self.observation_shape = (600, 800, 3)
        self.observation_space = space.Box(low = np.zeros(self.observation_shape),
                                           high = np.zeros(self.observation_shape),
                                           dtype=np.float16)

        self.action_space = space.Discrete(6, )
        self.canvas = np.ones(self.observation_shape) * 1
        self.elements = []
        self.max_fuel = 1000

        self.y_min = int(self.observation_shape[0] * 0.1)
        self.x_min = 0
        self.y_max = int(self.observation_shape[0] * 0.9)
        self.x_max = self.observation_shape[1]

    def draw_elements_on_canvas(self):

        self.canvas = np.ones(self.observation_shape) * 1

        for elem in self.elements:
            elem_shape = elem.icon.shape
            x, y = elem.x, elem.y
            self.canvas[y : y + elem_shape[1], x:x + elem_shape[0]] = elem.icon

        text = f'Fuel Left: {self.fuel_left} | Reward: {self.ep_return}'

        self.canvas = cv2.putText(self.canvas, text, (10, 20), font, 0.8, (0,0,0), 1, cv2.LINE_AA)

    def reset(self):
        self.fuel_left = self.max_fuel
        self.ep_return = 0

        self.bird_count = 0
        self.fuel_count = 0




class Point(object):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        self.x = 0
        self.y = 0
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max

        self.name = name

    def set_position(self, x, y):
        self.x = self.clamp(x, self.x_min, self.x_max - self.icon_w)
        self.y = self.clamp(y, self.y_min, self.y_max - self.icon_h)

    def get_position(self):
        return (self.x, self.y)

    def move(self, del_x, del_y):
        self.x += del_x
        self.y += del_y

        self.x = self.clamp(self.x, self.x_min, self.x_max - self.icon_W)
        self.y = self.clamp(self.y, self.y_min, self.y_max - self.icon_h)

    def clamp(self, n, minn, maxn):
        return max(min(maxn, n), minn)

class Chopper(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Chopper, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread('chopper.png') / 255
        self.icon_w = 64
        self.icon_h = 64
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))


class Bird(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Bird, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread('bird.png') / 255
        self.icon_w = 32
        self.icon_h = 32
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))

class Fuel(Point):
    def __init__(self, name, x_max, x_min, y_max, y_min):
        super(Fuel, self).__init__(name, x_max, x_min, y_max, y_min)
        self.icon = cv2.imread("fuel.png") / 255.0
        self.icon_w = 32
        self.icon_h = 32
        self.icon = cv2.resize(self.icon, (self.icon_h, self.icon_w))